# Known Problem:
- **Unresolved**
- **Partially Resolved**
    1. Certain websites cause the error of, specified here: https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests, which is a connection error. 
        - this appears to be windows specific as this was not a relevant issue when testing code in ubuntu 22.04, some forum reflect similar message
        - FAILED SOLUTION: first response in that stackoverflow page, still kept the stuff bc it seemed like better QOL
        - ATTEMPTED SOLUTION: used try and except. NOTE: this solution is not ideal as it does not resolve the issue, it just ignores it and moves on.
- **Resolved**
    1. The search "what is a taylor series" returns SSLCertVerification Error.
        - this resolution does not consider the security of the websites being accessed, potentially use try except to have the request verify when possible and not break when cannot verify
        - ATTEMPTED SOLUTION: used try except to use verify=False when ssl error occurs. Nested try except to ensure that both connection error and ssl error are handled.

    

# Part 1: Data Acquisition & Preparation

## Collecting & Preprocessing Our Data

In [4]:
#imports
import requests #send HTTP requests to web pages and retrieve their content
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from urllib.parse import urlparse, urljoin
import pandas as pd
from nltk.corpus import stopwords
import http.client
import re

In [2]:
import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
#uncomment if you cannot properly download punkt

nltk.download('punkt')
#ssl._create_default_https_context = ssl._create_default_https_context
#uncomment if you cannot properly download punkt

#remove filler words from an example column
nltk.download("stopwords")

nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/william/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/william/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/william/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#Working implementation

# Define the search query and retrieve search results
search_query = input("What would you like to search up?")
search_engine_url = "https://www.google.com/search?q="

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Number of search result pages to retrieve (each page typically has 10 results)
num_pages = 5

search_results = []

def verify_ssl_certificate(url):
    try:
        response = requests.get(url, verify=True, headers=headers)
        return True
    except requests.exceptions.SSLError:
        return False

for page in range(num_pages):
    # Calculate the start index for the current page
    start_index = page * 10

    try:
        # Make a GET request to retrieve the search result page
        response = requests.get(search_engine_url + search_query + f"&start={start_index}", headers=headers)
        response.raise_for_status()  # Raise an exception for non-2xx status codes

    except requests.exceptions.RequestException as e:
        print("Error occurred for page", page+1)
        print("Exception:", e)
        continue

    # Parse and extract information from the search result page
    soup = BeautifulSoup(response.content, "html.parser")

    for result in soup.select("div.g"):
        title_element = result.select_one("h3")
        if title_element:
            title = title_element.get_text()
            url_element = result.find("a")
            if url_element:
                url = url_element["href"]
                parsed_url = urlparse(url)
                if parsed_url.scheme == "":
                    # Relative URL, convert it to absolute URL
                    base_url = response.url
                    url = urljoin(base_url, url)

                # Verify SSL certificate before making the GET request
                if verify_ssl_certificate(url):
                    try:
                        print("Trying:", url)
                        result_response = requests.get(url, headers=headers)
                        result_response.raise_for_status()  # Raise an exception for non-2xx status codes
                        result_soup = BeautifulSoup(result_response.content, "html.parser")
                        text = result_soup.get_text()
                        text = ' '.join(text.split())
                        search_results.append({"Title": title, "URL": url, "Text": text})
                    except http.client.IncompleteRead:
                        print("Error: Incomplete read for:", url)
                        continue
                    except Exception as e:
                        print("Error: Failed to retrieve content for:", url)
                        print("Exception:", e)
                        continue

                # Make a GET request to the URL of the search result
                else:
                    print("SSL certificate verification failed for:", url)
                    continue
                    

print("Grabbed search results")

# Process the search results as desired

Trying: https://en.wikipedia.org/wiki/Taylor_series
Trying: https://mathworld.wolfram.com/TaylorSeries.html
Trying: https://www.mathsisfun.com/algebra/taylor-series.html
Trying: https://machinelearningmastery.com/a-gentle-introduction-to-taylor-series/
Trying: https://byjus.com/maths/taylor-series/
Trying: https://www.britannica.com/science/Taylor-series
Trying: https://www.math.net/taylor-series
Trying: https://brilliant.org/wiki/taylor-series-approximation/
Trying: https://www.britannica.com/science/Taylor-series
Trying: https://people.math.sc.edu/girardi/m142/handouts/10sTaylorPolySeries.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Trying: https://www.math.net/taylor-series
Trying: https://web.ma.utexas.edu/users/m408s/CurrentWeb/LM11-11-2.php
Trying: https://betterexplained.com/articles/taylor-series/
Trying: https://calcworkshop.com/sequences-series/taylor-series/
Trying: https://www.geeksforgeeks.org/taylor-series/
Trying: https://math.libretexts.org/Bookshelves/Calculus/Calculus_3e_(Apex)/08%3A_Sequences_and_Series/8.07%3A_Taylor_Polynomials
Trying: https://brilliant.org/wiki/taylor-series-approximation/
Trying: https://users.math.msu.edu/users/magyarp/Math133/11.10-Taylor-Series.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Trying: https://math.libretexts.org/Bookshelves/Calculus/Calculus_3e_(Apex)/08%3A_Sequences_and_Series/8.07%3A_Taylor_Polynomials
Trying: https://xaktly.com/TaylorSeries.html
Trying: http://hyperphysics.phy-astr.gsu.edu/hbase/tayser.html
Trying: https://www.mathworks.com/help/symbolic/sym.taylor.html
Trying: http://www.math.smith.edu/~rhaas/m114-00/chp4taylor.pdf


...

Trying: https://ncatlab.org/nlab/show/Taylor+series
Trying: https://sites.millersville.edu/bikenaga/calculus2/taylor-series/taylor-series.html
Trying: https://blog.cambridgecoaching.com/where-do-taylor-series-come-from-and-why-do-we-learn-about-them
Trying: https://complex-analysis.com/content/taylor_series.html
Trying: https://www.geeksforgeeks.org/taylor-series/
Trying: https://www.cuemath.com/taylor-series-formula/
Trying: https://education.ti.com/html/t3_free_courses/calculus84_online/mod24/mod24_lesson3.html
Trying: https://users.math.msu.edu/users/magyarp/Math133/11.10-Taylor-Series.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Trying: https://xaktly.com/TaylorSeries.html
Trying: http://hyperphysics.phy-astr.gsu.edu/hbase/tayser.html
Trying: https://www.mathworks.com/help/symbolic/sym.taylor.html
Trying: http://www.math.smith.edu/~rhaas/m114-00/chp4taylor.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Trying: https://ncatlab.org/nlab/show/Taylor+series
Trying: https://sites.millersville.edu/bikenaga/calculus2/taylor-series/taylor-series.html
Trying: https://blog.cambridgecoaching.com/where-do-taylor-series-come-from-and-why-do-we-learn-about-them
Trying: https://complex-analysis.com/content/taylor_series.html
Trying: https://www.efunda.com/math/taylor_series/taylor_series.cfm
Trying: https://www.whitman.edu/mathematics/calculus_late_online/section13.10.html
Trying: https://mathcs.org/analysis/reals/funseq/taylor.html
Trying: https://myweb.uiowa.edu/pbreheny/7110/wiki/taylor-series.html
Trying: https://www.sciencedirect.com/topics/engineering/taylor-series-expansion
Error: Failed to retrieve content for: https://www.sciencedirect.com/topics/engineering/taylor-series-expansion
Exception: 403 Client Error: Forbidden for url: https://www.sciencedirect.com/topics/engineering/taylor-series-expansion
Trying: https://x-engineer.org/taylor-series-approximation/
Trying: https://www.kristaking

In [5]:
df = pd.DataFrame(search_results)
df_bart = pd.DataFrame(search_results)

# Extract domain names from URLs and set as the index for the dataframe
tld_pattern = re.compile(r'\.[a-zA-Z]+$')
df['Domain'] = df['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df.insert(0, 'Domain', df.pop('Domain'))

df_bart['Domain'] = df_bart['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df_bart.insert(0, 'Domain', df_bart.pop('Domain'))
#Handles the "Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER" exception
df_bart['Text'] = df_bart['Text'].str.replace('\ufffd', '')

#unfiltered dataframe
print(len(df))
df.head()

46


,Domain,Title,URL,Text
0,en.wikipedia,Taylor series,https://en.wikipedia.org/wiki/Taylor_series,Taylor series - Wikipedia Jump to content Main...
1,mathworld.wolfram,Taylor Series -- from Wolfram MathWorld,https://mathworld.wolfram.com/TaylorSeries.html,Taylor Series -- from Wolfram MathWorld TOPICS...
2,mathsisfun,Taylor Series,https://www.mathsisfun.com/algebra/taylor-seri...,Taylor Series Advanced Show Ads Hide Ads About...
3,machinelearningmastery,A Gentle Introduction to Taylor Series,https://machinelearningmastery.com/a-gentle-in...,A Gentle Introduction to Taylor Series - Machi...
4,byjus,Taylor Series (Proof and Examples),https://byjus.com/maths/taylor-series/,Taylor Series (Proof and Examples) Login Study...


In [6]:
stop_words = stopwords.words("english") #english filler words

def preprocess(text):
    '''
    Input a text block and filter out unneeded characters
    returns a filtered text block in the form of a str
    Function filters whitespace, numbers, special characters, stopwords; handles case normalization 
    '''
    # Remove special characters and numbers
    cleaned_text = re.sub(r"[^a-zA-Z]+", " ", text)
    
    # Convert to lowercase and split into words
    words = cleaned_text.lower().split()
    
    # Remove stop words and single-character words
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1]
    lemmatized_text = " ".join([WordNetLemmatizer().lemmatize(word) for word in filtered_words])
    
    return lemmatized_text

def bart_preprocess(text):

    words = text.split()
    lemmatized_text = " ".join([WordNetLemmatizer().lemmatize(word) for word in words])

    return lemmatized_text


In [7]:
example = df["Text"][0]
processed_example = preprocess(example)

print("length of unfiltered text: ", len(example))
print("length of filtered text: ", len(processed_example))

length of unfiltered text:  51507
length of filtered text:  27771


In [8]:
#Properly using the apply() function onto a pandas column
#removes both filler words and special characters
df["Text"] = df["Text"].apply(preprocess)
df_bart["Text"] = df_bart["Text"].apply(bart_preprocess)


In [9]:
print(len(df["Text"][0]))

27771


In [10]:
#filtered text for first row entry
df["Text"][0]

'taylor series wikipedia jump content main menu main menu move sidebar hide navigation main pagecontentscurrent eventsrandom articleabout wikipediacontact usdonate contribute helplearn editcommunity portalrecent changesupload file language language link top page across title search search create accountlog personal tool create account log page logged editor learn contributionstalk content move sidebar hide top definition example history analytic function approximation error convergence toggle approximation error convergence subsection generalization list maclaurin series common function toggle list maclaurin series common function subsection exponential function natural logarithm geometric series binomial series trigonometric function hyperbolic function polylogarithmic function elliptic function calculation taylor series toggle calculation taylor series subsection first example second example third example taylor series definition taylor series several variable toggle taylor series se

# Part 2: EDA, Feature Engineering, and Model Development

### Future Optimizations

1. Ignore website navigation for websites like (wikipedia, stackoverflow, etc.)


### Tasks

1. Research into transformer, BART model, to better understand model set up and how to get highest performance
2. Experiment with BART input (lemmatize/not, capitalization, etc.) to find best performance 
3. Try to implement BART model with recursion as per the drawn diagram 
4. Research into paralellization of transformer models with HuggingFace 

### Notes

1. To actually make project work, we need to create a few metrics that we can train a DL model on. Maybe, linguistic complexity (formulaic), sentiment analysis, text length, etc.

In [11]:
import torch
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame called 'df' containing your data
# Split the data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Split the train data further into train and validation sets (80% train, 20% validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Print the sizes of each dataset
print("Train set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Test set size:", len(test_data))

Train set size: 28
Validation set size: 8
Test set size: 10


In [12]:
test_data.reset_index().drop(["index"], axis = 1)
val_data.reset_index().drop(["index"], axis = 1)
train_data.reset_index().drop(["index"], axis = 1)

,Domain,Title,URL,Text
0,people.math.sc,Commonly Used Taylor Series,https://people.math.sc.edu/girardi/m142/handou...,pdf obj length filter flatedecode stream ko jl...
1,web.ma.utexas,Applications of Taylor Polynomials,https://web.ma.utexas.edu/users/m408s/CurrentW...,application taylor polynomial prev next home f...
2,brilliant,Taylor Series Approximation | Brilliant Math &...,https://brilliant.org/wiki/taylor-series-appro...,taylor series approximation brilliant math sci...
3,cuemath,Taylor Series Formula,https://www.cuemath.com/taylor-series-formula/,expert math tutoring uk boost score cuemathgra...
4,math,Taylor series,https://www.math.net/taylor-series,taylor series home calculus series taylor seri...
5,users.math.msu,Taylor Series,https://users.math.msu.edu/users/magyarp/Math1...,pdf obj length filter flatedecode stream qp nf...
6,socratic,"What does the ""a"" means in the Taylor Series e...",https://socratic.org/questions/what-does-the-a...,mean taylor series expansion socratic calculus...
7,education.ti,Module 24 - Power Series - Lesson 3,https://education.ti.com/html/t3_free_courses/...,module power series lesson module power series...
8,mathworld.wolfram,Taylor Series -- from Wolfram MathWorld,https://mathworld.wolfram.com/TaylorSeries.html,taylor series wolfram mathworld topic algebra ...
9,calcworkshop,Taylor Series,https://calcworkshop.com/sequences-series/tayl...,taylor series please click redirected within s...


In [13]:
#Using Bart
# !pip install transformers
# !pip install torch
from transformers import BartForConditionalGeneration, AutoTokenizer

In [14]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
# device = "cuda:0" if torch.cuda.is_available() else "cpu"


In [19]:
len(df_bart["Text"][0])

51230

In [21]:
inputs = tokenizer([df_bart["Text"][0]],max_length = 1024,return_tensors="pt")
# model = model.to(device)
# outputs = model(**inputs)


In [27]:
summary_ids = model.generate(inputs["input_ids"], num_beams=1, min_length=0, max_length=30)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'Taylor series is a series of equations. The Taylor series is the product of the Taylor and the logarithm.'